### Import packages and load the data

In [20]:
import os
import numpy as np                                    
import pandas as pd

In [21]:
main_dir = "C:\\Users\\jzhou202\\Comcast\\EBI CB Applied Analytics - Documents\\CX\\NPS\\Detractor Promoter Journeys"
data_dir = main_dir + "/Data"
output_dir = main_dir + "/Output"
os.chdir(data_dir)
print(os.getcwd())

C:\Users\jzhou202\Comcast\EBI CB Applied Analytics - Documents\CX\NPS\Detractor Promoter Journeys\Data


In [22]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [23]:
data = pd.read_csv('cluster_data_for_profile 2019-07-23.csv')

### Clean the data

In [24]:
data.isnull().sum()

CLUSTER_NUM                              0
SURVEY_DATE                              0
ACCOUNT_NUMBER                           0
PROMOTER_DETRACTOR                       0
FISCAL_MONTH_END_DATE                    0
ACTIVE_CLV                            8982
AUTO_PAY_ENROLLMENT                      0
PRODUCT_MIX                              0
HSD_TIER_NAME                            0
IVR_TOTAL_CNT_2_MNTHS                    0
IVR_RESOLVED_TOTAL_CNT_2_MNTHS           0
IVR_UNRESOLVED_TOTAL_CNT_2_MNTHS         0
TTS_TOTAL_TICKETS_CNT_2_MNTHS            0
TRUCKROLL_TOTAL_CNT_CNT_2_MNTHS          0
HSD_TOTAL_GB_CNT_2_MNTHS                 0
HSD_DOWNLOAD_GB_CNT_2_MNTHS              0
HSD_UPLOAD_GB_CNT_2_MNTHS                0
INBOUND_CALLS_CNT_2_MNTHS            33807
OUTBOUND_CALLS_CNT_2_MNTHS           33807
TOT_ET_1_OUTAGE_CNT_2_MNTHS              0
IVR_TOTAL_CNT_12_MNTHS                   0
IVR_RESOLVED_TOTAL_CNT_12_MNTHS          0
IVR_UNRESOLVED_TOTAL_CNT_12_MNTHS        0
TTS_TOTAL_T

In [25]:
# Drop surveys with no information in Rosetta and BC_SUBS (about 900)
data_with_no_disc = data[data['PRODUCT_MIX']!='NO PRODUCTS'].copy()

In [26]:
# Drop surveys with no information in Rosetta and BC_SUBS (about 900)
data_clean = data_with_no_disc.dropna(
    subset=['TRUCKROLL_TOTAL_CNT_CNT_12_MNTHS', 'TENURE']).copy()

In [27]:
data.shape

(93835, 56)

In [28]:
data_clean.shape

(93469, 56)

In [29]:
# Impute with 0 for below variables
cols_0 = ['PROMO_IND_2_MONTH_GREATER_30',
          'PROMO_IND_2_MONTH_LESS_30', 'PROMO_IND_12_MONTH_GREATER_30', 'PROMO_IND_12_MONTH_LESS_30', 'EBILL']
data_clean[cols_0] = data_clean[cols_0].fillna(0)

In [30]:
# Impute with mean value for below variables
cols_mean = ['ACTIVE_CLV', 'ATHENA_SITE_FIRM_REVENUE',
             'ATHENA_SITE_EST_HEADCOUNT']
data_clean[cols_mean] = data_clean[cols_mean].fillna(
    data_clean[cols_mean].mean())

In [31]:
# Impute with PBB for Athena segment
data_clean['ATHENA_MKTG_SUBSEGMENT'] = data_clean['ATHENA_MKTG_SUBSEGMENT'].fillna('PBB')

In [32]:
# Impute with mean value for the voice usage variables when BCV_IND==1
cols_mean_2 = ['INBOUND_CALLS_CNT_2_MNTHS',
               'OUTBOUND_CALLS_CNT_2_MNTHS', 'INBOUND_CALLS_CNT_12_MNTHS', 'OUTBOUND_CALLS_CNT_12_MNTHS']
for i in cols_mean_2:
    data_clean.loc[data_clean['BCV_IND'] == 1, [i]
                   ] = data_clean[i].fillna(data_clean[i].mean())

In [33]:
# Impute with 0 for the voice usage variables when BCV_IND==0
data_clean[cols_mean_2] = data_clean[cols_mean_2].fillna(0)

In [34]:
df = data_clean.copy()

### Create Profile Analysis Script

In [35]:
df.head()

,CLUSTER_NUM,SURVEY_DATE,ACCOUNT_NUMBER,PROMOTER_DETRACTOR,FISCAL_MONTH_END_DATE,ACTIVE_CLV,AUTO_PAY_ENROLLMENT,PRODUCT_MIX,HSD_TIER_NAME,IVR_TOTAL_CNT_2_MNTHS,IVR_RESOLVED_TOTAL_CNT_2_MNTHS,IVR_UNRESOLVED_TOTAL_CNT_2_MNTHS,TTS_TOTAL_TICKETS_CNT_2_MNTHS,TRUCKROLL_TOTAL_CNT_CNT_2_MNTHS,HSD_TOTAL_GB_CNT_2_MNTHS,HSD_DOWNLOAD_GB_CNT_2_MNTHS,HSD_UPLOAD_GB_CNT_2_MNTHS,INBOUND_CALLS_CNT_2_MNTHS,OUTBOUND_CALLS_CNT_2_MNTHS,TOT_ET_1_OUTAGE_CNT_2_MNTHS,IVR_TOTAL_CNT_12_MNTHS,IVR_RESOLVED_TOTAL_CNT_12_MNTHS,IVR_UNRESOLVED_TOTAL_CNT_12_MNTHS,TTS_TOTAL_TICKETS_CNT_12_MNTHS,TRUCKROLL_TOTAL_CNT_CNT_12_MNTHS,HSD_TOTAL_GB_CNT_12_MNTHS,HSD_DOWNLOAD_GB_CNT_12_MNTHS,HSD_UPLOAD_GB_CNT_12_MNTHS,INBOUND_CALLS_CNT_12_MNTHS,OUTBOUND_CALLS_CNT_12_MNTHS,TOT_ET_1_OUTAGE_CNT_12_MNTHS,BCV_IND,BCI_IND,VIDEO_IND,TOTAL_MRC_AMT,TENURE,VOICELINE_CNT,ATHENA_SITE_FIRM_REVENUE,ATHENA_SITE_EST_HEADCOUNT,ATHENA_MKTG_SUBSEGMENT,ATHENA_MKTG_INDUSTRY,PROMO_IND_2_MONTH_GREATER_30,PROMO_IND_2_MONTH_LESS_30,PROMO_IND_12_MONTH_GREATER_30,PROMO_IND_12_MONTH_LESS_30,INTERNET_BUCKETS,NEW_PRODUCT_IND,COMPETITOR_IND,EBILL,MRC_2_MNTHS_CHANGE,MRC_12_MNTHS_CHANGE,MRM_2_MNTHS_CHANGE,MRM_12_MNTHS_CHANGE,CHURN_IND,UPGRADE_IND,DOWNGRADE_IND
0,1,2017-02-08,8497950380222439,-1,2017-02-21,14657.480000,0,VIDEO/HSD/CDV,BS STARTER,0,0,0,0,0,801.692,707.592,94.099,205.0,104.0,4,2,0,2,1,0,3842.309,3286.307,556.005,1297.0,982.0,4,1,1,1,196.20,69,3,76000.0,2.0,HOSPITALITY,"HOTELS, ROOMING HOUSES, CAMPS, AND OTHER LODGI...",0.0,0.0,0.0,0.0,INTERNET_LOW,0,0,0.0,0.0,43.61,2.15,44.02,0,0,0
1,1,2017-07-13,8396900140194750,0,2017-07-21,18647.640000,0,VIDEO/HSD/CDV,BS DELUXE,0,0,0,0,0,143.481,120.905,22.576,423.0,297.0,0,5,2,3,3,2,790.834,663.001,127.834,2753.0,1844.0,9,1,1,1,166.65,186,4,189000.0,2.0,PBB,PERSONAL SERVICES,0.0,0.0,0.0,0.0,INTERNET_MID,0,0,0.0,0.0,-61.99,0.00,-63.75,0,0,0
2,3,2018-01-11,8299600860899772,0,2018-01-21,15165.206874,0,VIDEO/HSD/CDV,BS DELUXE 150 HSD,0,0,0,0,0,703.332,609.717,93.616,3567.0,426.0,1,11,1,10,8,0,2989.068,2602.224,386.845,17786.0,2135.0,3,1,1,1,603.30,58,4,2165529.0,85.0,PBB,EATING AND DRINKING PLACES,0.0,0.0,0.0,1.0,INTERNET_MID,0,0,0.0,1.0,-69.65,0.35,-74.66,0,0,0
3,6,2018-09-21,8773500760016235,1,2018-09-21,12974.420000,1,VIDEO/HSD/CDV,BS DELUXE 25 HSD,1,0,1,1,0,16.561,13.881,2.680,774.0,830.0,2,10,1,9,8,2,97.898,87.345,10.552,3720.0,3392.0,6,1,1,1,163.25,146,2,193156.0,2.0,HBB,"AUTOMOTIVE REPAIR, SERVICES AND PARKING",0.0,0.0,0.0,0.0,INTERNET_LOW,0,0,0.0,0.0,7.00,-2.30,8.16,0,0,0
4,1,2019-01-03,8155600395770525,-1,2019-01-21,16343.080000,0,VIDEO/HSD/CDV,BS STARTER,0,0,0,0,0,325.447,316.316,9.131,130.0,8.0,0,0,0,0,0,0,699.607,677.071,22.535,871.0,213.0,4,1,1,1,186.15,78,2,250000.0,2.0,PBB,CONSTRUCTION - SPECIAL TRADE CONTRACTORS,0.0,0.0,0.0,0.0,INTERNET_LOW,0,0,1.0,0.0,3.40,0.66,2.27,0,0,1


In [36]:
# Dimensions to profile
dimensions = [
    'PRODUCT_MIX',
    'HSD_TIER_NAME',
    'INTERNET_BUCKETS',
    'ATHENA_MKTG_SUBSEGMENT',
    'ATHENA_MKTG_INDUSTRY'
]

In [37]:
# Variable to profile
variables = ['PROMOTER_DETRACTOR',
             'ACTIVE_CLV',
             'EBILL',
             'AUTO_PAY_ENROLLMENT',
             'BCV_IND',
             'BCI_IND',
             'VIDEO_IND',
             'PROMO_IND_2_MONTH_GREATER_30',
             'PROMO_IND_2_MONTH_LESS_30',
             'PROMO_IND_12_MONTH_GREATER_30',
             'PROMO_IND_12_MONTH_LESS_30',
             'NEW_PRODUCT_IND',
             'COMPETITOR_IND',
             'IVR_RESOLVED_TOTAL_CNT_2_MNTHS',
             'IVR_UNRESOLVED_TOTAL_CNT_2_MNTHS',
             'TTS_TOTAL_TICKETS_CNT_2_MNTHS',
             'TRUCKROLL_TOTAL_CNT_CNT_2_MNTHS',
             'HSD_TOTAL_GB_CNT_2_MNTHS',
             'INBOUND_CALLS_CNT_2_MNTHS',
             'OUTBOUND_CALLS_CNT_2_MNTHS',
             'TOT_ET_1_OUTAGE_CNT_2_MNTHS',
             'IVR_RESOLVED_TOTAL_CNT_12_MNTHS',
             'IVR_UNRESOLVED_TOTAL_CNT_12_MNTHS',
             'TTS_TOTAL_TICKETS_CNT_12_MNTHS',
             'TRUCKROLL_TOTAL_CNT_CNT_12_MNTHS',
             'HSD_TOTAL_GB_CNT_12_MNTHS',
             'INBOUND_CALLS_CNT_12_MNTHS',
             'OUTBOUND_CALLS_CNT_12_MNTHS',
             'TOT_ET_1_OUTAGE_CNT_12_MNTHS',
             'TOTAL_MRC_AMT',
             'TENURE',
             'VOICELINE_CNT',
             'ATHENA_SITE_FIRM_REVENUE',
             'ATHENA_SITE_EST_HEADCOUNT',
             'MRC_2_MNTHS_CHANGE',
             'MRC_12_MNTHS_CHANGE',
             'MRM_2_MNTHS_CHANGE',
             'MRM_12_MNTHS_CHANGE',
             'CHURN_IND',
             'UPGRADE_IND',
             'DOWNGRADE_IND'
             ]

In [38]:
# Continious Variables Profile
table = pd.pivot_table(df, values=variables, index=[
                    'CLUSTER_NUM'], aggfunc=np.mean, margins = True, margins_name = 'Total')
table

,ACTIVE_CLV,ATHENA_SITE_EST_HEADCOUNT,ATHENA_SITE_FIRM_REVENUE,AUTO_PAY_ENROLLMENT,BCI_IND,BCV_IND,CHURN_IND,COMPETITOR_IND,DOWNGRADE_IND,EBILL,HSD_TOTAL_GB_CNT_12_MNTHS,HSD_TOTAL_GB_CNT_2_MNTHS,INBOUND_CALLS_CNT_12_MNTHS,INBOUND_CALLS_CNT_2_MNTHS,IVR_RESOLVED_TOTAL_CNT_12_MNTHS,IVR_RESOLVED_TOTAL_CNT_2_MNTHS,IVR_UNRESOLVED_TOTAL_CNT_12_MNTHS,IVR_UNRESOLVED_TOTAL_CNT_2_MNTHS,MRC_12_MNTHS_CHANGE,MRC_2_MNTHS_CHANGE,MRM_12_MNTHS_CHANGE,MRM_2_MNTHS_CHANGE,NEW_PRODUCT_IND,OUTBOUND_CALLS_CNT_12_MNTHS,OUTBOUND_CALLS_CNT_2_MNTHS,PROMOTER_DETRACTOR,PROMO_IND_12_MONTH_GREATER_30,PROMO_IND_12_MONTH_LESS_30,PROMO_IND_2_MONTH_GREATER_30,PROMO_IND_2_MONTH_LESS_30,TENURE,TOTAL_MRC_AMT,TOT_ET_1_OUTAGE_CNT_12_MNTHS,TOT_ET_1_OUTAGE_CNT_2_MNTHS,TRUCKROLL_TOTAL_CNT_CNT_12_MNTHS,TRUCKROLL_TOTAL_CNT_CNT_2_MNTHS,TTS_TOTAL_TICKETS_CNT_12_MNTHS,TTS_TOTAL_TICKETS_CNT_2_MNTHS,UPGRADE_IND,VIDEO_IND,VOICELINE_CNT
CLUSTER_NUM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,9296.595778,14.517264,3.221280e+06,0.318007,0.900494,0.056615,0.020793,0.251784,0.004529,0.336948,1913.959384,440.869981,5.749413,1.067870,0.800508,0.160994,0.756451,0.097584,2.583105,1.002178,1.530012,0.850729,0.012901,18.828502,3.844290,0.192287,0.008441,0.093055,0.002882,0.023264,75.702443,149.120571,3.544057,0.713011,0.704021,0.072468,1.011254,0.161337,0.011117,0.368652,0.069380
1,17837.015680,8.189546,1.429249e+06,0.301488,0.970396,1.000000,0.012174,0.263459,0.014879,0.318415,1334.898439,303.046490,4324.728741,874.657730,1.289005,0.257275,1.180329,0.149874,8.347561,2.081419,7.091389,1.868149,0.037565,3123.561080,637.833636,0.128657,0.008232,0.167266,0.004097,0.046609,70.910454,217.483675,7.093604,1.473662,0.800386,0.086609,1.683246,0.270300,0.001971,0.671575,2.750493
2,14966.441669,10.170723,2.270893e+06,0.296556,0.953020,0.701199,0.017741,0.258939,0.013470,0.313585,1518.499608,350.999048,2918.758515,592.375980,1.158681,0.309095,1.000000,0.427641,5.854493,1.393812,4.526550,1.245185,0.031484,2097.518081,430.413289,0.140722,0.008870,0.141926,0.003504,0.045776,73.488145,199.182108,5.997646,1.301922,0.808191,0.145212,1.466736,0.480644,0.005640,0.589717,1.923507
3,15656.208178,11.507684,2.235568e+06,0.280752,0.974586,0.840150,0.018346,0.234135,0.016241,0.313233,2089.326684,483.121192,6554.909368,1362.758398,4.389173,0.784060,11.509023,1.560752,7.301674,0.346235,6.066795,-0.000253,0.058346,5041.550891,1100.849864,-0.101654,0.008872,0.145714,0.003308,0.042105,78.666466,252.024612,8.850526,1.814737,2.457444,0.282707,9.800000,1.367068,0.003910,0.700000,2.919549
4,16121.506816,10.425461,2.064335e+06,0.290247,0.966270,0.790206,0.013742,0.243774,0.011160,0.301741,1715.564378,404.007923,4456.297295,912.643492,2.513534,0.363788,5.033980,0.221704,6.499860,1.064742,5.433878,0.827745,0.047889,3301.969397,687.824607,0.052886,0.010411,0.152744,0.003748,0.038894,77.219872,225.702767,7.437578,1.379695,1.564004,0.101607,4.822020,0.341384,0.003665,0.651537,2.469559
5,15228.757739,10.352450,2.024248e+06,0.288009,0.958183,0.747992,0.028385,0.259208,0.022431,0.301025,1562.240011,360.190317,3644.876772,752.998464,1.435060,0.522570,1.365965,1.000000,7.576101,1.273934,6.286382,1.123163,0.036140,2670.548555,560.302992,0.097065,0.009000,0.141097,0.006092,0.050678,75.099834,210.561440,6.323041,1.486153,0.857796,0.266685,1.800332,0.993769,0.006646,0.617973,2.188729
6,16012.829988,9.999139,2.046263e+06,0.272411,0.974753,0.834431,0.022320,0.252104,0.015551,0.296561,1805.170486,419.201767,5491.083374,1123.705864,3.303147,0.668862,7.188072,1.000000,7.493386,0.181389,6.044232,-0.106014,0.057080,4239.574224,880.613618,-0.027259,0.008233,0.143798,0.002927,0.048482,78.751006,242.147291,8.508050,1.740578,1.874863,0.236553,6.562569,1.074643,0.003659,0.687523,2.788328
7,15216.429074,9.947199,1.742144e+06,0.279438,0.973931,0.815759,0.038957,0.263327,0.030170,0.298477,1776.807893,435.910373,5357.960237,1119.957444,2.191857,1.310779,3.067663,4.471295,9.944815,2.461845,8.173193,2.440085,0.049795,4096.144260,871.049663,-0.116872,0.0093

In [39]:
# Continious Variables standard deviation
table_sd = pd.pivot_table(df, values=variables, index=[
                    'CLUSTER_NUM'], aggfunc=np.std)
table_sd

,ACTIVE_CLV,ATHENA_SITE_EST_HEADCOUNT,ATHENA_SITE_FIRM_REVENUE,AUTO_PAY_ENROLLMENT,BCI_IND,BCV_IND,CHURN_IND,COMPETITOR_IND,DOWNGRADE_IND,EBILL,HSD_TOTAL_GB_CNT_12_MNTHS,HSD_TOTAL_GB_CNT_2_MNTHS,INBOUND_CALLS_CNT_12_MNTHS,INBOUND_CALLS_CNT_2_MNTHS,IVR_RESOLVED_TOTAL_CNT_12_MNTHS,IVR_RESOLVED_TOTAL_CNT_2_MNTHS,IVR_UNRESOLVED_TOTAL_CNT_12_MNTHS,IVR_UNRESOLVED_TOTAL_CNT_2_MNTHS,MRC_12_MNTHS_CHANGE,MRC_2_MNTHS_CHANGE,MRM_12_MNTHS_CHANGE,MRM_2_MNTHS_CHANGE,NEW_PRODUCT_IND,OUTBOUND_CALLS_CNT_12_MNTHS,OUTBOUND_CALLS_CNT_2_MNTHS,PROMOTER_DETRACTOR,PROMO_IND_12_MONTH_GREATER_30,PROMO_IND_12_MONTH_LESS_30,PROMO_IND_2_MONTH_GREATER_30,PROMO_IND_2_MONTH_LESS_30,TENURE,TOTAL_MRC_AMT,TOT_ET_1_OUTAGE_CNT_12_MNTHS,TOT_ET_1_OUTAGE_CNT_2_MNTHS,TRUCKROLL_TOTAL_CNT_CNT_12_MNTHS,TRUCKROLL_TOTAL_CNT_CNT_2_MNTHS,TTS_TOTAL_TICKETS_CNT_12_MNTHS,TTS_TOTAL_TICKETS_CNT_2_MNTHS,UPGRADE_IND,VIDEO_IND,VOICELINE_CNT
CLUSTER_NUM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4637.058236,63.950918,2.404516e+07,0.465718,0.299350,0.231114,0.142697,0.434053,0.067149,0.472683,5033.419806,1398.337645,71.132642,10.448319,1.942009,0.558825,1.137112,0.443131,29.686081,14.653632,30.716223,14.822261,0.112853,588.200178,100.589628,0.783258,0.091489,0.290520,0.053611,0.150745,59.895562,138.012693,5.278210,1.631728,1.249705,0.371915,1.698949,0.612012,0.104854,0.482456,0.314242
1,9088.275143,13.539226,4.218584e+06,0.458913,0.169495,0.000000,0.109664,0.440518,0.121072,0.465871,3479.235720,778.601471,6729.290299,1470.504149,5.684746,0.801326,1.258573,0.526590,33.289519,19.075647,33.309079,17.838799,0.190146,5324.867010,1149.559964,0.794360,0.090357,0.373220,0.063875,0.210803,47.449225,107.006939,9.737133,3.201153,1.382819,0.416430,2.443786,0.854525,0.044353,0.469649,2.000514
2,8865.776950,38.299186,2.523300e+07,0.456751,0.211602,0.457745,0.132011,0.438064,0.115279,0.463962,3718.060351,868.360606,5752.340250,1163.596176,2.016335,0.780055,0.000000,0.985275,49.829739,19.347140,49.848896,19.002537,0.174628,4467.577319,908.590941,0.792527,0.093767,0.348984,0.059095,0.209004,53.211358,175.989468,9.899215,2.897693,1.326237,0.548637,1.945956,1.228136,0.074889,0.491898,2.080150
3,9807.020820,86.098355,2.181770e+07,0.449400,0.157389,0.366494,0.134209,0.423489,0.126409,0.463843,4531.360413,1049.415982,12139.862408,2585.801782,5.053345,1.468776,6.349356,2.393968,64.130333,28.519615,62.135580,29.456156,0.234414,10245.864305,2396.591260,0.821145,0.093781,0.352846,0.057427,0.200844,58.978141,147.226084,11.482339,4.345400,2.346521,0.749443,6.737075,2.260817,0.062411,0.458292,2.690032
4,9417.581402,34.369300,1.938537e+07,0.453895,0.180542,0.407179,0.116423,0.429376,0.105055,0.459032,3779.810291,1030.577885,8430.225808,1729.419408,3.060601,0.899820,1.042498,0.627924,53.705589,22.687530,50.046138,21.682540,0.213540,6403.165859,1381.205592,0.808574,0.101504,0.359756,0.061107,0.193350,58.365625,192.043959,11.833986,3.009480,1.781608,0.446919,3.417950,0.947641,0.060427,0.476503,2.396910
5,9291.586346,36.999109,1.306030e+07,0.452867,0.200184,0.434196,0.166083,0.438230,0.148092,0.458735,3515.758085,807.238187,6922.819858,1457.393375,2.327412,0.980761,1.263386,0.000000,34.515081,27.555571,35.147274,26.419706,0.186650,5464.280225,1182.593313,0.805479,0.094448,0.348145,0.077822,0.219356,55.262691,117.549869,8.725160,3.290532,1.359134,0.640502,2.365919,1.094656,0.081259,0.485917,2.267457
6,9808.029400,19.108312,1.447577e+07,0.445241,0.156889,0.371727,0.147735,0.434260,0.123740,0.456783,4365.161421,1105.242889,9697.724668,1991.679296,6.391965,1.148342,3.887312,0.000000,53.158240,27.355941,52.299495,26.929490,0.232017,9230.540536,1707.912676,0.818914,0.090368,0.350917,0.054029,0.214801,59.776828,151.758829,11.404483,3.771083,2.048203,0.610541,5.355797,1.277545,0.060384,0.463546,2.626144
7,9827.698866,18.789534,5.957582e+06,0.448789,0.159364,0.387738,0.193521,0.440503,0.171080,0.457657,3913.378339,1143.234140,10767.388194,2101.860570,3.090838,1.827073,2.205351,2.215776,44.351188,47.277337,42.498694,46.473849,0

In [40]:
# Categorical Variables Profile
dims = {}
for i in dimensions:
    dims['metrics_{}'.format(i)] = pd.crosstab(df.CLUSTER_NUM, df[i], normalize='index',
                                                   margins=True, margins_name="Total")
    print(dims)

{'metrics_PRODUCT_MIX': PRODUCT_MIX  CDV ONLY  HSD ONLY   HSD/CDV  VIDEO ONLY  VIDEO/CDV  VIDEO/HSD  VIDEO/HSD/CDV
CLUSTER_NUM                                                                               
0            0.007755  0.573977  0.044126    0.090379   0.001372   0.277450       0.004941
1            0.012638  0.000812  0.314860    0.000039   0.017005   0.000618       0.654029
2            0.010787  0.175820  0.222307    0.026009   0.010349   0.097793       0.456935
3            0.004361  0.078647  0.215639    0.013083   0.008571   0.071579       0.608120
4            0.006663  0.111768  0.226951    0.017573   0.009828   0.081203       0.546015
5            0.007754  0.142204  0.231238    0.023262   0.011216   0.087926       0.496400
6            0.003659  0.082876  0.223198    0.013355   0.008416   0.069887       0.598610
7            0.006151  0.092560  0.207088    0.014353   0.006444   0.081722       0.591681
Total        0.009019  0.163231  0.220458    0.025474   0.010410  

{'metrics_PRODUCT_MIX': PRODUCT_MIX  CDV ONLY  HSD ONLY   HSD/CDV  VIDEO ONLY  VIDEO/CDV  VIDEO/HSD  VIDEO/HSD/CDV
CLUSTER_NUM                                                                               
0            0.007755  0.573977  0.044126    0.090379   0.001372   0.277450       0.004941
1            0.012638  0.000812  0.314860    0.000039   0.017005   0.000618       0.654029
2            0.010787  0.175820  0.222307    0.026009   0.010349   0.097793       0.456935
3            0.004361  0.078647  0.215639    0.013083   0.008571   0.071579       0.608120
4            0.006663  0.111768  0.226951    0.017573   0.009828   0.081203       0.546015
5            0.007754  0.142204  0.231238    0.023262   0.011216   0.087926       0.496400
6            0.003659  0.082876  0.223198    0.013355   0.008416   0.069887       0.598610
7            0.006151  0.092560  0.207088    0.014353   0.006444   0.081722       0.591681
Total        0.009019  0.163231  0.220458    0.025474   0.010410  

In [41]:
# Output to Excel
writer = pd.ExcelWriter(output_dir + "/cluster profiles.xlsx",
                        engine='xlsxwriter')


table.to_excel(writer,
                 sheet_name="cluster profiles")

table_sd.to_excel(writer,
                 sheet_name="cluster std")


for i in dims.keys():
    dims[i].to_excel(writer, sheet_name="{}".format(i))

writer.save()